# N(z) Global

_Autores: Andreia Dourado, Bruno Moraes_

__Descrição: Passo a passo para gerar o N(z) dos outputs gerados ao rodar TPZ para o catálogo completo do DP02.__

### 1. Bibliotecas:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tables_io
import rail
import qp
import glob 

from rail.core.data import TableHandle, PqHandle, ModelHandle, QPHandle, DataHandle, Hdf5Handle
from rail.core.stage import RailStage

from qp.ensemble import Ensemble
from matplotlib import gridspec
from qp import interp

from rail.estimation.algos.naive_stack import NaiveStackSummarizer
from scipy.interpolate import UnivariateSpline

%matplotlib inline
%reload_ext autoreload
%autoreload 

### 2. Lendo os arquivos de output:

In [ ]:
output_files = glob.glob('/home/andreia.dourado/outputs_nz/*')

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

### 3. N(z) individuais para cada arquivo de output com o _NaiveStackSummarizer_:


In [ ]:
#for i in range(n_files+1):
for i,pdfs_file in enumerate(output_files[0:10]):
    #pdfs_file='/home/andreia.dourado/outputs_nz/objectTable_tract_2907*'
    pdfs_data = DS.read_file('pdfs_data', QPHandle, pdfs_file)
    zgrid = pdfs_data().metadata()['xvals'].ravel()
    photoz_mode = pdfs_data().mode(grid=zgrid)
    ensemble = DS.add_data('ensemble',pdfs_data(), QPHandle)
    
    stacker = NaiveStackSummarizer.make_stage(zmin=0.0, zmax=max(photoz_mode), nzbins=300, nsamples=20, hdf5_groupname=None, output="Naive_samples_part"+f'{i}'+".hdf5", single_NZ="NaiveStack_NZ_part"+f'{i}'+".hdf5")
    naive_results = stacker.summarize(pdfs_data)

### 4. Gerando o N(z)

#### 4.1 Fazendo o somátorio para os arquivos gerados no passo anterior:

sum -> ['data']['yvals']
bins -> ['meta']['xvals']

In [ ]:
arrays = {}

for i in range(301):
    array=[]
    for j in range(6):
        hist =tables_io.read('/home/andreia.dourado/outputs_nz/NaiveStack_NZ_part'+f'{j}'+'.hdf5')
        array.append(hist['data']['yvals'][0][i])
        name = 'array'+str(i)
        arrays[name] = array
    
     
somas=[]        
for i in range(301):
        soma = sum(arrays['array'+f'{i}']) 
        somas.append(soma)

data = {'counts': somas}

#### 4.2 Definindo os bins para o gráfico:

__Escolhendo um dos arquivos de output:__

In [ ]:
pdfs_file = '/home/andreia.dourado/outputs_nz/objectTable_tract_2907_DC2_2_2i_runs_DP0_2_v23_0_1_PREOPS-905_step3_3_20220309T040429Z-part0.hdf5'

In [ ]:
pdfs_data = DS.read_file('pdfs_data', QPHandle, pdfs_file)

In [ ]:
zgrid = pdfs_data().metadata()['xvals'].ravel()

#### 4.3 Plotando o N(z) global:

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(zgrid, data['counts'], color = 'blue', label = 'z$_{spec}$ PDF')